> pip install SpeechRecognition

In [1]:
import speech_recognition as sr
sr.__version__

'3.8.1'

In [2]:
r = sr.Recognizer()

### Recording or Uploading Audio File

In [3]:
harvard = sr.AudioFile('Audio 1.wav')
with harvard as source:
    audio = r.record(source)

In [4]:
type(audio)

speech_recognition.AudioData

### Doing it with Google

I have an observation here, that the free google API embeded here in SpeechRecognition library is only taking a few seconds or possibly, the first second of the audio.

In [5]:
r.recognize_google(audio)

'I am going to the'

In [6]:
r.recognize_google(audio, show_all=True)

{'alternative': [{'transcript': 'I am going to the', 'confidence': 0.83460408},
  {'transcript': 'I am going to the back'},
  {'transcript': 'I am going to the WhatsApp'},
  {'transcript': 'I am going to'},
  {'transcript': 'I am going to bus'}],
 'final': True}

### Doing it with IBM

IBM plan coming with free tier is 500 Minutes per Month.

In [7]:
!pip install ibm-watson

In [8]:
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [9]:
apikey = 'jXJZ0DFtm0iLg-XinOxqIRgbHlT-376eS3TfNPlFq37i'
url = 'https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/cabe9e09-1272-475f-a3e1-77c4c50ce8d2'

In [10]:
# Setup Service
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

In [11]:
# Perform conversion
with open('Audio 1.mp3', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel').get_result()

In [12]:
res

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'I am going to the bar but suddenly I remembered that I have a pending task to complete the audit ',
     'confidence': 0.93}]}]}

In [13]:
text = res['results'][0]['alternatives'][0]['transcript']
text

'I am going to the bar but suddenly I remembered that I have a pending task to complete the audit '

In [14]:
confidence = res['results'][0]['alternatives'][0]['confidence']
confidence

0.93

### SR with Sphinx and Wit.ai

sudo apt-get install -qq python python-dev python-pip build-essential swig libpulse-dev

In [18]:
!pip install pocketsphinx

  Using cached pocketsphinx-0.1.15.tar.gz (29.1 MB)
  ERROR: Command errored out with exit status 1:
   command: /home/saifbinkhaki/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/setup.py'"'"'; __file__='"'"'/tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-98d7vp0t
       cwd: /tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/
  Complete output (186 lines):
  running bdist_wheel
  running build_ext
  building 'sphinxbase._sphinxbase' extension
  swigging deps/sphinxbase/swig/sphinxbase.i to deps/sphinxbase/swig/sphinxbase_wrap.c
  s

Failed to build pocketsphinx
    Running setup.py install for pocketsphinx ... error
    ERROR: Command errored out with exit status 1:
     command: /home/saifbinkhaki/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/setup.py'"'"'; __file__='"'"'/tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-p2c_2kqp/install-record.txt --single-version-externally-managed --compile --install-headers /home/saifbinkhaki/anaconda3/include/python3.8/pocketsphinx
         cwd: /tmp/pip-install-w_9vgqdn/pocketsphinx_aeb63d75c8cd431ba0985523930937c1/
    Complete output (186

sudo apt update


sudo apt install pocketsphinx

In [19]:
#!/usr/bin/env python3

from pprint import pprint
import speech_recognition as sr

# obtain path to "english.wav" in the same folder as this script
from os import path

AUDIO_FILE = path.join("Audio 1.wav")
# AUDIO_FILE = path.join(path.dirname(path.realpath(__file__)), "french.aiff")
# AUDIO_FILE = path.join(path.dirname(path.realpath(__file__)), "chinese.flac")

# use the audio file as the audio source
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # read the entire audio file

# recognize speech using Sphinx
try:
    print("Sphinx thinks you said " + r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

# recognize speech using Wit.ai
WIT_AI_KEY = "A26KM7ZXBWPFETJQDSPN4YQFCYH62J2W"  # Wit.ai keys are 32-character uppercase alphanumeric strings
try:
    print("Wit.ai recognition results:")
    pprint(r.recognize_wit(audio, key=WIT_AI_KEY, show_all=True))  # pretty-print the recognition result
except sr.UnknownValueError:
    print("Wit.ai could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Wit.ai service; {0}".format(e))

ModuleNotFoundError: No module named 'pocketsphinx'

### SR with Azure

!pip install azure-cognitiveservices-speech

!pip install azure

In [16]:
import azure.cognitiveservices.speech as speechsdk

def from_file():
    speech_config = speechsdk.SpeechConfig(subscription="9289c64dc9ef4e79b070a73960c613a4", region="eastus")
    audio_input = speechsdk.AudioConfig(filename="Audio 1.wav")
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
    
    result = speech_recognizer.recognize_once_async().get()
    print(result.text)

from_file()

I am going to the bar, but suddenly I remembered that I have a pending task to complete the audit.


### SR with Houndify

In [17]:
# recognize speech using Houndify
HOUNDIFY_CLIENT_ID = "iOuUT9lsQAI3nO8E7_vx8g=="  # Houndify client IDs are Base64-encoded strings
HOUNDIFY_CLIENT_KEY = "VMxp_FhmxtLNyTVqA4_eZspCUR4BwMzwu99VW96Sn3PNQW1T_NkYeiETqW6z4OuwMq8k5X03B21-B6y0TsLJuQ=="  # Houndify client keys are Base64-encoded strings
try:
    print("Houndify recognition results:")
    pprint(r.recognize_houndify(audio, client_id=HOUNDIFY_CLIENT_ID, client_key=HOUNDIFY_CLIENT_KEY, show_all=True))
except sr.UnknownValueError:
    print("Houndify could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Houndify service; {0}".format(e))

Houndify recognition results:
{'ASRUsage': [{'ASRComponent': 'Audio',
               'AudioLength': 5.92,
               'ComponentUniqueID': '80800d1e-2876-4cc6-91dd-9033d1dc9c9c',
               'IncrementalAudioLength': 5.92}],
 'AllResults': [{'AutoListen': False,
                 'CommandKind': 'SpeechToTextOnlyCommand',
                 'FormattedTranscription': 'I am going to the bar but suddenly '
                                           'I remember that I have a pending '
                                           'task to complete the audit',
                 'NativeData': {'FormattedTranscription': 'I am going to the '
                                                          'bar but suddenly I '
                                                          'remember that I '
                                                          'have a pending task '
                                                          'to complete the '
                                             